# Deep learning binary classifier
## used to detect sentence boundary

1. Data preparation and feature extraction

In [26]:
import time
import collections
import nltk
from __future__ import print_function
from six.moves import xrange
import six.moves.cPickle as pickle
import random
t0 = time.time()

en = "/home/work/ssCat/sss.en"
tokens = []  #store all the tokes
boundaries = set()   #store boundary index
boundaries_token = set()  #store tokens which is a sentence boundary
known = 0.9 #only register 90% of words as known
sents = []
with open(en) as inputen:
    offset = 0
    for sent in inputen:
        #nltk tokenization
        sent = nltk.word_tokenize(sent.decode("utf-8")) 
        if len(sent) >=1:
            sents.append(sent)
            boundaries_token.add(sent[-1])
            tokens.extend(sent)
            offset += len(sent)
            boundaries.add(offset-1)
#frequent words will assaign small number
vocabulary_size = int(len(set(tokens))*known)
count = []
count.extend(collections.Counter(tokens).most_common(vocabulary_size - 1))
words = [a for a,b in count]
word2id = {w:i for i,w in enumerate(words)} 

#save dic
def save_word():
    pickle.dump(word2id,open("word2id.dict","w"))
fdic = {}

def get_feature(tokens,i):
    """The following features are used for English:
    boundary token
    previous word of boundary token
    next word of boundary token
    boolean if previous word is digit
    boolean if the first letter of the boundary token is capital
    boolean if the first letter of the next word is capital
    boolean if the first letter of previous word is capital
    boolean if the first letter of the previous 2 word is capital"""  
    return ['punct=%s'%tokens[i] if tokens[i] in words else "UNK",\
            'prepunct=%s'%tokens[i-1] if tokens[i] in words else "UNK",\
            'nextpunct=%s'%tokens[i+1] if tokens[i] in words else "UNK",\
            'prepunctdigit=%s'%tokens[i-1].isdigit(),\
            'punctCapi=%s'%tokens[i][0].isupper(),\
            'next1letterCapi=%s'%tokens[i+1][0].isupper(),\
            'pre1LetterCap=%s'%tokens[i-1][0].isupper(),\
            'pre2LetterCap=%s'%tokens[i-2][0].isupper()]

#make a feature dic     

def get_feature_from_token():
    feature_sets = []
    for i in xrange(2, len(tokens)-2):
        feature_sets.extend(get_feature(tokens,i))
    return feature_sets
feature_sets = get_feature_from_token()        
        
print (len(feature_sets),(len(tokens)-4)*8,type(feature_sets))
feature_sets = collections.Counter(feature_sets).most_common(len(feature_sets))
feature = [f for f,i in feature_sets]
f2id = {f:i+1 for i,f in enumerate(feature)}

#print f2id['prepunctdigit=True']

feature_sets = [(get_feature(tokens,i)) for i in xrange(2, len(tokens)-2)]
fid_sets = [[f2id[f] for f in l] for l in feature_sets]
#print train_fid_sets[:5]
labels = [i in boundaries for i in xrange(2,len(tokens)-2)]

            
            


1696496 1696496 <type 'list'>


In [31]:
print(fid_sets[:9])

[[6697, 38, 417, 1, 4, 2, 3, 6], [397, 6664, 1401, 35, 4, 2, 3, 5], [1358, 410, 14, 1, 4, 2, 3, 5], [13, 1415, 8622, 1, 4, 7, 3, 5], [8238, 15, 164, 1, 8, 2, 3, 5], [162, 8723, 3156, 1, 4, 2, 9, 5], [3299, 163, 20, 1, 4, 2, 3, 6], [19, 3158, 3586, 1, 4, 2, 3, 5], [3620, 21, 12, 1, 4, 2, 3, 5]]


In [28]:
#filter out too many false example
num_to_select = 1  # set the number to select here.
def balance_feature():
    index_collecter = []
    feature = []
    label = []
    for i in range(len(labels)):
        if labels[i] == False:
            index_collecter.append(i)
        if labels[i] == True:
            selected_index = random.sample(index_collecter, num_to_select)
            for j in selected_index:
                feature.append(fid_sets[j])
                label.append(labels[j])
            feature.append(fid_sets[i])
            label.append(labels[i])
    return feature,label
f,l = balance_feature()
print (len(f),len(l))
pickle.dump(f,open("equal_class_feature","w"))
pickle.dump(l,open("equal_class_label","w"))

19998 19998


In [29]:
def rename_label(train_labels):
    new_labels = []
    for i in train_labels:
        if i == True:
            new_labels.extend([1])
        elif i == False:
            new_labels.extend([2])
    return new_labels
y = rename_label(l)
pickle.dump(l,open("equal_class_num_label","w"))
#print (y[:40])
#print (train_labels[:40])

In [33]:
#from sklearn.preprocessing import LabelEncoder
#y = LabelEncoder().fit_transform(train_labels)
y_train = labels[:int(len(labels)*0.9)]
y_dev = labels[int(len(labels)*0.9):]
x_train = fid_sets[:int(len(fid_sets)*0.9)]
x_dev = fid_sets[int(len(fid_sets)*0.9):]

In [16]:
#pickle.dump(word2id,open("word2id","w"))
#pickle.dump(f2id,open("f2id","w"))
pickle.dump(x_train,open("train_data","w"))
pickle.dump(y_train,open("train_label","w"))
pickle.dump(x_dev,open("dev_data","w"))
pickle.dump(y_dev,open("dev_label","w"))

In [1]:
import numpy as np
import six.moves.cPickle as pickle

x_train = pickle.load(open("train_data"))
y_train = pickle.load(open("train_label"))
x_dev = pickle.load(open("dev_data"))
y_dev = pickle.load(open("dev_label"))

In [17]:
nf = 8
nl = 2
def reformat(dataset, labels):
    dataset = np.array(dataset).reshape((-1, 8)).astype(np.float32)
    # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(2) == np.array(labels)[:,None]).astype(np.float32)
    return dataset, labels
#only once
#x_train, y_train = reformat(x_train,y_train)
#x_dev, y_dev = reformat(x_dev,y_dev)

print('Training set', len(x_train), len(y_train))
#print(x_train.dtype,y_train.dtype,x_dev.dtype,y_dev.dtype)
#print('Validation set', x_dev.shape, y_dev.shape)

Training set 190855 190855


In [3]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions,1) == np.argmax(labels,1))
          / predictions.shape[0])

In [2]:
def true_class(p,l):
    tp = 0
    ta = 0
    #p = np.ndarray.tolist(p)
    #l = np.ndarray.tolist(l)
    for x,y in zip (p,l):
        if x == y == True:
            tp+=1
        if y == True:
            ta+=1
            
    return float(tp/ta)

print(true_class(y_dev,y_dev))
        

1.0


In [10]:
#print (y_train[:35])
#print (x_train[:35])
print (y_dev[:50])
#print (x_dev[:35])

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]


In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
model = LinearSVC()
model.fit(x_train, y_train)
predict = model.predict(x_dev)
print(true_class(predict,y_dev))
train_score = model.score(x_train, y_train)
test_score = model.score(x_dev, y_dev)
print(train_score,test_score)

0.0
0.929690078856 0.949073419154


In [8]:
### two layers neural network
import tensorflow as tf
import numpy as np

N, D, H, C = 190855, 8, 8, 2 

graph = tf.Graph()
with graph.as_default():

    x = tf.placeholder(tf.float32, shape = [N,D])
    y = tf.placeholder(tf.float32, shape = [N,C])
    x_d = tf.constant(x_dev)
    y_d = tf.constant(y_dev)
    dtype = tf.float32
    w1 = tf.Variable(1e-4 * np.random.randn(D,H).astype(np.float32))
    w2 = tf.Variable(1e-4 * np.random.randn(H,C).astype(np.float32))

    def model(data):
        #1-2 layer
        a = tf.matmul(data,w1) 
        a_relu = tf.nn.relu(a)

        #2-3 layer
        scores = tf.matmul(a_relu,w2)
        return scores

    #loss
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(model(x), y))

    #for tensorboard
    loss_summary = tf.scalar_summary("loss",loss)
    w1_hist = tf.histogram_summary('w1',w1)
    w2_hist = tf.histogram_summary('w2',w2)

    #optimize
    learning_rate = 1e-5
    optimizer = tf.train.AdadeltaOptimizer(learning_rate).minimize(loss)

    #prediction
    p_train = tf.nn.softmax(model(x_train))
    p_dev = tf.nn.softmax(model(x_d))

    num_steps = 400
    batch_size = 1000
    with tf.Session() as sess:
        merged = tf.merge_all_summaries()
        writer = tf.train.SummaryWriter('/home/work/sstmp/ss_log',sess.graph)
        #sess.run(tf.initialize_all_tables())
        tf.initialize_all_variables().run()
        for step in range(num_steps):
            feed_dict = {x : x_train, y : y_train}
            _, l, predictions = sess.run([optimizer, loss, p_train], feed_dict=feed_dict)
            if (step % 100 == 0):
                print('Minibatch loss at step %d: %f' % (step, l))
                print('Training accuracy: %.1f%%' % accuracy(predictions, y_train))
                print('Training true accuracy: %.1f%%' % true_class(predictions, y_train))
                print('Validation accuracy: %.1f%%' % accuracy(p_dev.eval(),y_dev))
                print('Validation true accuracy: %.1f%%' % true_class(p_dev.eval(),y_dev))

Minibatch loss at step 0: 0.693350
Training accuracy: 4.9%
Training true accuracy: 0.0%
Validation accuracy: 3.3%
Validation true accuracy: 0.0%
Minibatch loss at step 100: 0.692180
Training accuracy: 95.1%
Training true accuracy: 0.0%
Validation accuracy: 96.7%
Validation true accuracy: 0.0%
Minibatch loss at step 200: 0.638872
Training accuracy: 95.1%
Training true accuracy: 0.0%
Validation accuracy: 96.7%
Validation true accuracy: 0.0%
Minibatch loss at step 300: 0.433518
Training accuracy: 95.1%
Training true accuracy: 0.0%
Validation accuracy: 96.7%
Validation true accuracy: 0.0%
